# Pyomo constraints

In [1]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_style = r'..\style_1.css'
css_file = css_style
HTML(open(css_file, "r").read())

In [2]:
from pyomo.environ import *

## Declaring constraints

The most direct way of declaration is:

In [3]:
model = ConcreteModel()
model.x = Var([1,2], initialize = 1)
model.diff = Constraint(expr = model.x[2] - model.x[1] <= 7.5)

In [4]:
model.diff.pprint()

diff : Size=1, Index=None, Active=True
    Key  : Lower : Body        : Upper : Active
    None :  -Inf : x[2] - x[1] :   7.5 :   True


It is more intuitive to use `rule` instead of `expr`. For example:

In [5]:
def diff_rule(model):
    return model.x[2] - model.x[1] <= 7.5
model.diff2 = Constraint(rule = diff_rule)
model.diff2.pprint()

diff2 : Size=1, Index=None, Active=True
    Key  : Lower : Body        : Upper : Active
    None :  -Inf : x[2] - x[1] :   7.5 :   True


To indicate to which index the restriction applies :

In [6]:
N = [1,2,3]
a = {1:1, 2:3.1, 3:4.5}
b = {1:1, 2:2.9, 3:3.1}

model.y = Var(N, within = NonNegativeReals, initialize = 0)

def CoverConstr_rule(model,i):
    return a[i] * model.y[i] >= b[i]
model.CoverConstr = Constraint(N, rule = CoverConstr_rule)
model.CoverConstr.pprint()

CoverConstr : Size=3, Index=CoverConstr_index, Active=True
    Key : Lower : Body       : Upper : Active
      1 :   1.0 :       y[1] :  +Inf :   True
      2 :   2.9 : 3.1 * y[2] :  +Inf :   True
      3 :   3.1 : 4.5 * y[3] :  +Inf :   True


## Types of constraints

There are three types of constraints, although it is better to say that there are only two and one that can be expressed like two of other type.

#### 1) Unequality constraints


$$expr_{1} \leq expr_{2} \qquad or\qquad expr_{1} \geq expr_{2}$$

#### 2) Equality constraints


$$ expr_{1} = expr_{2}$$

#### 3) Bound constraints


$$ lb \leq expr_{1} \leq ub$$

Sometimes, there exists the need of not defining a constraint for a certain case in an index. For example, for a time period (Normally the first or the last). For these cases, we use `Constraint.Skip`.

In [7]:
model.TimePeriods = Set(initialize = [1,2,3,4,5], ordered = True)

In [8]:
LastTimePeriod = model.TimePeriods[-1]
model.StartTime = Var(model.TimePeriods, initialize = 1)
def Pred_rule(model,t):
    if t == LastTimePeriod:
        return Constraint.Skip
    else:
        return model.StartTime[t] <= model.StartTime[t+1]
model.Pred = Constraint(model.TimePeriods, rule = Pred_rule)
model.Pred.pprint()

Pred : Size=4, Index=TimePeriods, Active=True
    Key : Lower : Body                        : Upper : Active
      1 :  -Inf : StartTime[1] - StartTime[2] :   0.0 :   True
      2 :  -Inf : StartTime[2] - StartTime[3] :   0.0 :   True
      3 :  -Inf : StartTime[3] - StartTime[4] :   0.0 :   True
      4 :  -Inf : StartTime[4] - StartTime[5] :   0.0 :   True


## Obtaining information from constraints

Some useful commands to check if what you're writing is what you meant to write.

In [9]:
m = ConcreteModel()
m.x = Var(initialize = 1)
m.y = Var(initialize = 1)
m.c1 = Constraint(expr = m.y - m.x <= 7.5)
m.c2 = Constraint(expr = -2.5 <= m.y - m.x)
m.c3 = Constraint(expr = -3 <= m.y - m.x <= 7)

In [10]:
print(value(m.c1.body))

0


In [11]:
print(m.c1.body)

y - x


In [12]:
print(m.c1.lslack())
print(m.c1.uslack())

-inf
7.5
